In [195]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats

## Exploratory Data Analysis
This notebook is an exploration of the `udemy_courses` dataset as downloaded from Kaggle.

In [196]:
df = pd.read_csv('udemy_courses.csv')
df.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5,2017-01-18T20:58:58Z,Business Finance
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39.0,2017-03-09T16:34:20Z,Business Finance
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5,2016-12-19T19:26:30Z,Business Finance
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3.0,2017-05-30T20:07:24Z,Business Finance
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2.0,2016-12-13T14:57:18Z,Business Finance


In [197]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3678 entries, 0 to 3677
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   course_id            3678 non-null   int64  
 1   course_title         3678 non-null   object 
 2   url                  3678 non-null   object 
 3   is_paid              3678 non-null   bool   
 4   price                3678 non-null   int64  
 5   num_subscribers      3678 non-null   int64  
 6   num_reviews          3678 non-null   int64  
 7   num_lectures         3678 non-null   int64  
 8   level                3678 non-null   object 
 9   content_duration     3678 non-null   float64
 10  published_timestamp  3678 non-null   object 
 11  subject              3678 non-null   object 
dtypes: bool(1), float64(1), int64(5), object(5)
memory usage: 319.8+ KB


> ### Observations:
- No null values
- `course_id` (`int64`) is not numerically meaningful
- `is_paid` can be binarized
- `published_timestamp` is type `object`, not `datetime`

> ### Actions
At this time, I'll binarize `is_paid`, convert `course_id` to `object`, and convert `published_timestamp` to `datetime`.

In [198]:
# Binarize is_paid column
print(df.is_paid.value_counts())
df.is_paid = df.is_paid.apply(lambda x: 1 if x is True else 0)
df.is_paid.value_counts()

True     3368
False     310
Name: is_paid, dtype: int64


1    3368
0     310
Name: is_paid, dtype: int64

In [199]:
# Set course_id to string dtype
df.course_id = df.course_id.astype(str)
df.course_id.dtype

dtype('O')

In [200]:
# Set published_timestamp to datetime dtype
df.published_timestamp = pd.to_datetime(df.published_timestamp)
df.published_timestamp.dtype

datetime64[ns, UTC]

In [201]:
# fig = px.scatter(df, x='num_subscribers', y='num_reviews')
# fig.show()

In [202]:
# fig = px.scatter(df, x='num_subscribers', y='price')
# fig.show()

In [203]:
df.subject.value_counts()

Web Development        1200
Business Finance       1195
Musical Instruments     680
Graphic Design          603
Name: subject, dtype: int64

## Analyzing Courses By Subject

In [204]:
df.describe()

,is_paid,price,num_subscribers,num_reviews,num_lectures,content_duration
count,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000,3678.000000
mean,0.915715,66.049483,3197.150625,156.259108,40.108755,4.094517
std,0.277852,61.005755,9504.117010,935.452044,50.383346,6.053840
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,20.000000,111.000000,4.000000,15.000000,1.000000
50%,1.000000,45.000000,911.500000,18.000000,25.000000,2.000000
75%,1.000000,95.000000,2546.000000,67.000000,45.750000,4.500000
max,1.000000,200.000000,268923.000000,27445.000000,779.000000,78.500000


> Based on the descriptive statistics above, it seems likely the data contains outliers across all features (except `price`). I will save a copy of the data with the outliers removed.

In [205]:
def drop_outliers_from_column(df, col):
    """Remove column outliers from a DataFrame."""
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3-Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_sans_ol = df.loc[(df[col] > lower_bound) & (df[col] < upper_bound)]
    return df_sans_ol

In [206]:
df_sans_ol = drop_outliers_from_column(df, 'num_subscribers')
df_sans_ol = drop_outliers_from_column(df_sans_ol, 'num_reviews')
df_sans_ol = drop_outliers_from_column(df_sans_ol, 'num_lectures')
df_sans_ol = drop_outliers_from_column(df_sans_ol, 'content_duration')
df_sans_ol.describe()

,is_paid,price,num_subscribers,num_reviews,num_lectures,content_duration
count,2420.000000,2420.000000,2420.000000,2420.000000,2420.000000,2420.000000
mean,0.947521,57.580579,938.855785,17.680165,23.260744,2.060255
std,0.223038,54.552974,1226.695052,21.432961,14.296461,1.255952
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,20.000000,50.000000,3.000000,12.000000,1.000000
50%,1.000000,40.000000,416.000000,9.000000,20.000000,1.500000
75%,1.000000,70.000000,1355.250000,25.000000,30.000000,3.000000
max,1.000000,200.000000,6134.000000,101.000000,76.000000,5.500000


> The maximum values are now more reasonably distanced from the rest of the data.

In [207]:
# Create DataFrame subject subsets

web_dev_df = df_sans_ol[df_sans_ol.subject == 'Web Development']
bus_fin_df = df_sans_ol[df_sans_ol.subject == 'Business Finance']
music_df = df_sans_ol[df_sans_ol.subject == 'Musical Instruments']
graph_des_df = df_sans_ol[df_sans_ol.subject == 'Graphic Design']

In [208]:
def mean_feature_by_subject(column):
    """Print the average value of a DataFrame column for each subject."""
    
    print('Web Dev %s Mean:' % column, web_dev_df[column].mean())
    print('Business Finance %s Mean:' % column, bus_fin_df[column].mean())
    print('Musical Instr %s Mean:' % column, music_df[column].mean())
    print('Graphic Design %s Mean:' % column, graph_des_df[column].mean())

In [209]:
mean_feature_by_subject('price')

Web Dev price Mean: 63.05068226120858
Business Finance price Mean: 64.66483516483517
Musical Instr price Mean: 45.824175824175825
Graphic Design price Mean: 51.29711751662971


In [210]:
mean_feature_by_subject('num_subscribers')

Web Dev num_subscribers Mean: 1876.6783625730993
Business Finance num_subscribers Mean: 791.4362637362638
Musical Instr num_subscribers Mean: 469.6043956043956
Graphic Design num_subscribers Mean: 737.6585365853658


In [211]:
mean_feature_by_subject('num_reviews')

Web Dev num_reviews Mean: 32.97465886939571
Business Finance num_reviews Mean: 16.53956043956044
Musical Instr num_reviews Mean: 9.183150183150182
Graphic Design num_reviews Mean: 12.87139689578714


In [212]:
mean_feature_by_subject('num_lectures')

Web Dev num_lectures Mean: 25.567251461988302
Business Finance num_lectures Mean: 21.90989010989011
Musical Instr num_lectures Mean: 25.278388278388277
Graphic Design num_lectures Mean: 20.92017738359202


In [213]:
mean_feature_by_subject('content_duration')

Web Dev content_duration Mean: 2.2474983755685516
Business Finance content_duration Mean: 2.083553113553114
Musical Instr content_duration Mean: 1.9467643467643465
Graphic Design content_duration Mean: 1.9376570583887662


> On average:
- With the exception of `price`, `Web Development` has the highest values across all quantitative columns.
- `Web Development` is the most popular `subject` with over twice as many subscribers and reviews as its closest contender.
- `Musical Instruments`, which happens to be the `subject` with the lowest price, contains the least popular courses, showing the lowest numbers of subscribers and reviews.


In [217]:
paid_df = df_sans_ol[df_sans_ol.is_paid == 1]
free_df = df_sans_ol[df_sans_ol.is_paid == 0]

In [218]:
paid_df.describe()

,is_paid,price,num_subscribers,num_reviews,num_lectures,content_duration
count,2293.0,2293.000000,2293.000000,2293.000000,2293.000000,2293.000000
mean,1.0,60.769734,866.641954,16.692979,23.587440,2.082432
std,0.0,54.286650,1167.866732,20.607059,14.434176,1.262390
min,1.0,20.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,20.000000,43.000000,2.000000,13.000000,1.000000
50%,1.0,40.000000,360.000000,9.000000,20.000000,1.500000
75%,1.0,75.000000,1266.000000,23.000000,30.000000,3.000000
max,1.0,200.000000,6134.000000,101.000000,76.000000,5.500000


In [219]:
free_df.describe()

,is_paid,price,num_subscribers,num_reviews,num_lectures,content_duration
count,127.0,127.0,127.000000,127.000000,127.000000,127.000000
mean,0.0,0.0,2242.685039,35.503937,17.362205,1.659843
std,0.0,0.0,1506.922696,27.507538,9.850203,1.060416
min,0.0,0.0,1.000000,0.000000,5.000000,0.516667
25%,0.0,0.0,980.500000,12.000000,10.000000,1.000000
50%,0.0,0.0,2084.000000,30.000000,15.000000,1.500000
75%,0.0,0.0,3193.500000,52.000000,23.000000,2.000000
max,0.0,0.0,5769.000000,101.000000,69.000000,5.000000


In [232]:
fig = px.histogram(df_sans_ol, x="price", y="num_subscribers", color='subject')
fig.show()

In [230]:
fig = px.histogram(df_sans_ol, x="price", color='subject')
fig.show()

> The most common price points are multiples of 25$ or so. Since the number of subscribers dips and peak similarly to the price point graph, it doesn't appear that price is a major factor in a course gaining subscribers.